In [ ]:
""" Initialize, import libraries and load data """

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

import statsmodels.api as sm

data_path = 'data/index_data/'  

# load data
with open(data_path + 'run4-mimicking.pkl', 'rb') as f:
    results = pickle.load(f)

trad_country_returns = results["trad_country_returns"]
csm_returns = results["csm_returns"]


This section provides mean-variance spanning tests (Huberman and Kandel, 1988). This methodology has previously been applied in the context of international diversification. Mean-variance spanning works by selecting a set of initial asset, commonly called benchmark assets. Next to that, there are test assets. Mean-variance spanning provides an answer to the question of whether the mean-variance efficient frontier of the test assets is the same as that of the benchmark assets.

Huberman and Kandel's (HK) approach is that of a regression:

Y = a + bX + e

Where Y is an N x t matrix of returns of test assets; X is a K x t matrix of returns of benchmark assets; a is 1 x K vector of intercepts coefficients and b a 1 x K vector of coefficients. [look at dimensions; do not yet match!]

The HK test looks at the intercepts - in the case where the two mean-variance frontiers are the same, the intercepts will also be 0. Or put differently, when the intercepts are not statistically significant from zero, we can infer that the mean-variance frontier of the test assets is not significantly different from that of the benchmark assets. This, in turn, implies that there are no diversification benefits from adding the test assets.



First we perform iterative mean spanning tests, where have as N benchmark assets the traditional country indices; and each time, we add a different test asset K to the mix. In these cases, K is always 1. The set of K test assets to draw from is the set of individual CSM country indices.

In [ ]:
spanning_results = {}
X = sm.add_constant(trad_country_returns)
for country in csm_returns.columns:
    y = csm_returns[country]
    model = sm.OLS(y, X).fit()
    spanning_results[country] = model

for test_asset, model in spanning_results.items():
    print(f"Results for CSM index country: {test_asset}")
    print(model.summary())
    print("\n")


The regression below presents the entire model. This is a multivariate model, meaning the lhs of the equation consists of all the test assets (N x t). In order to facilitate the regression tooling software, the data were transformed so that the dependent variable was one long vector of all the columns {r1, ..., rN} of the original matrix R.

R -> [r1, r2, r3]'

Its dimensions are now (N x t) x 1

On the righthand side, the benchmark asset returns were 'repeated' so as to properly match the dependent variable data and new dimensions. The block of R (N x K) was extended N times. So that the dimensions are now (t x N) x K. Previously the shape was N x K.



In [ ]:
# Combined model:
y_combined = csm_returns.values.flatten()
X_combined = np.tile(
    trad_country_returns.values, 
    (csm_returns.shape[1], 1)
)
X_combined = sm.add_constant(X_combined)

combined_model = sm.OLS(y_combined, X_combined).fit()
print(combined_model.summary())

# f_test_result = combined_model.f_test("const = 0")
# print(f_test_result.summary())

The model shows that the regression coefficient of the intercept is different from zero with statistical significance (p < 0.001). The result that not all coefficients are (near) zero suggests that the benchmark assets cannot span the total return variance of test assets and therefore, that there are diversification benefits of the CSM indices.



# Efficient frontier
We can also visualize the efficient frontiers for portfolios constructed using either country index. We construct the efficient frontier as follows. We find a set of weights w such that for various levels of returns, we minimize the volatility of the portfolio:

min (w'Sw)^(1/2)

s.t. sum(w) = 1
and 0 <= w_i <= 1

where w is an 1 x N vector of weights, for N number of countries and S is the N x N sample covariance matrix. The covariance matrix is constructed by simply taking the entire range of monthly returns for each asset. The returns are therefore in monthly terms, which is a bit odd to interpret but does not change the essence of the frontiers.

We perform this exercise twice, once for the CSM indices and again for the traditional country indices. We can then compare the shape of the two.

In [ ]:
from utils import create_efficient_frontier

csm_mean_returns = csm_returns.mean().values
csm_cov_matrix = csm_returns.cov()

trad_mean_returns = trad_country_returns.mean().values
trad_cov_matrix = trad_country_returns.cov()

# combined_returns = pd.concat([csm_returns, trad_country_returns], axis=1)
# combined_mean_returns = combined_returns.mean().values
# combined_cov_matrix = combined_returns.cov()

ef_csm = create_efficient_frontier(csm_mean_returns, csm_cov_matrix)
ef_trad = create_efficient_frontier(trad_mean_returns, trad_cov_matrix)
# ef_combined = create_efficient_frontier(combined_mean_returns, combined_cov_matrix)


In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(ef_csm.volatility * 100, ef_csm.returns * 100, label='Efficient Frontier csm', color='blue', linestyle='-')
plt.plot(ef_trad.volatility * 100, ef_trad.returns * 100, label='Efficient Frontier trad', color='red', linestyle='-')
# plt.plot(ef_combined.volatility * 100, ef_combined.returns * 100, label='Efficient Frontier combined', color='green', linestyle='--')
plt.xlabel('Portfolio Volatility (Std. Dev.)')
plt.ylabel('Portfolio Return')
plt.title('Efficient Frontier')
plt.legend()
plt.grid(True)
plt.show()



# Monte Carlo simulations
This section explores the diversification properties of the CSM indices versus their traditional country proponents using Monte Carlo simulations. For both index types (csm and trad) we randomly sample countries to construct portfolios. For each simulation run, we vary the number of countries we draw. So we randomly draw 2 countries, 3 countries, 5 countries, etc. Each draw, we calculate the equal-weighted portfolio returns of the sampled countries and calculate annualized (geometric) return, standard deviation of the returns and risk/reward ratio (the ratio of the first two metrics - called the Sharpe ratio in the graph, although it is technically not so). This is repeated 10.000 times. We repeat this procedure for specific numbers of countries (2, 3, 5, 7, 10, 13, 16) and for both index types. We finally proceed to aggregate the results by retaining only specific points from the obtained distribution. For each set of outcomes of 10.000 runs we select only the 2.5 percentile, 97.5 percentile and the median observations. 

The results are shown in the graphs below. The first panel shows only the median results for both index types, in one graph. The second panel also presents the lower (2.5 percentile) and upper (97.5 percentile) ranges of the distribution of the results. They are shown in separate plots for clarity. The third panel presents the results for the risk/reward ratio.

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)

simulations.plot_std_dev(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=[0.5], facet_plot=False)

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)

simulations.plot_std_dev(results_quantiles, metric="std_dev", n_runs=10000, list_of_quantiles=None, facet_plot=True)

In [ ]:
import simulations

# load data
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)

simulations.plot_std_dev(results_quantiles, metric="sharpe_ratio", n_runs=10000, list_of_quantiles=None, facet_plot=True)

We can visually interpret these results as follows. Let's first look at the change in standard deviation as the number of countries in portfolio increases. We are not directly interested in absolute levels (these are slightly higher on average for CSM indices), we want to look at the shape of each graph and compare that. So this graph, for the CSM indices, starts at around 6.8% (for 2 countries) and ends at 5.9%. This is a decrease of 14%. For the trad indices, the curve starts at 6% (for 2 countries) and ends at 5.4%. This is a decrease of 10%. Moreover, the curve for the traditional country indices seems to be slightly more concave, meaning it approaches the asymptote faster. 

These results suggests that the diversification benefits of traditional country indices are depleted faster. It takes fewer different countries in the index to obtain a decrease in portfolio risk similar to holding all countries. A flipside of this observation is that one could say that these traditional country indices are more alike. For the CSM indices, it takes more countries to approximate the portfolio standard deviation of the full set of countries. This suggests that CSM indices are more different from one another than traditional country indices are.

The statistics mentioned above in the text are summarized in the table below.

In [ ]:
results_quantiles = pd.read_csv('data/simulations_results_quantiles_n_runs_10000.csv', index_col=0)
csm_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
csm_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'csm') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

trad_median_stddev_start = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 2)]['std_dev']
trad_median_stddev_end = results_quantiles[(results_quantiles['dataset'] == 'trad') & (results_quantiles.index == 0.5) & (results_quantiles['num_countries'] == 16)]['std_dev']

pd.DataFrame({
    'CSM': 
    {
        'median_stddev_start (N=2)': csm_median_stddev_start.values[0],
        'median_stddev_end (N=16)': csm_median_stddev_end.values[0],
        'relative_change': csm_median_stddev_end.values[0] / csm_median_stddev_start.values[0] - 1
    },
    'Traditional': 
    {
        'median_stddev_start (N=2)': trad_median_stddev_start.values[0],
        'median_stddev_end (N=16)': trad_median_stddev_end.values[0],
        'relative_change': trad_median_stddev_end.values[0] / trad_median_stddev_start.values[0] - 1
    }
})
